In [162]:
import pandas as pd
import numpy as np
import ast
import gzip
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer


In [163]:
steam_games = pd.read_json('.\\data\\original\\steam_games.json.gz',compression='gzip',lines=True)


In [164]:
steam_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 11.9+ MB


In [165]:
#calculo el porcentaje de valores nulos en cada columna
100*steam_games.isna().sum()/len(steam_games)

publisher       80.004982
genres          76.045498
app_name        73.321433
title           75.021794
url             73.319773
release_date    75.035909
tags            73.455104
reviews_url     73.321433
specs           73.876043
price           74.463033
early_access    73.319773
id              73.321433
developer       76.058782
dtype: float64

In [166]:
#Borro todas las filas que estan completamente Nulas
steam_games = steam_games.dropna(how='all')

In [167]:
#vuelvo a calcular el porcentaje de valores nulos en cada columna
100*steam_games.isna().sum()/len(steam_games)

publisher       25.056792
genres          10.216275
app_name         0.006224
title            6.379337
url              0.000000
release_date     6.432239
tags             0.507235
reviews_url      0.006224
specs            2.084954
price            4.285047
early_access     0.000000
id               0.006224
developer       10.266065
dtype: float64

In [168]:
#cantidad de nulos por columna
steam_games.isna().sum()

publisher       8052
genres          3283
app_name           2
title           2050
url                0
release_date    2067
tags             163
reviews_url        2
specs            670
price           1377
early_access       0
id                 2
developer       3299
dtype: int64

In [169]:
steam_games.sample(10)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
95773,None,None,Kalen Chock Presents: Tips for Foliage,Kalen Chock Presents: Tips for Foliage,http://store.steampowered.com/app/519320/Kalen...,2016-10-03,"[Design & Illustration, Tutorial]",http://steamcommunity.com/app/519320/reviews/?...,None,2.99,0.0,519320.0,None
91378,GTGD,"[Indie, Simulation, Strategy]",Guild Commander,Guild Commander,http://store.steampowered.com/app/337220/Guild...,2015-01-23,"[Simulation, Strategy, Indie]",http://steamcommunity.com/app/337220/reviews/?...,"[Single-player, Steam Achievements]",4.99,0.0,337220.0,GTGD
95197,Richard Haraším,[Indie],A Week of Circus Terror,A Week of Circus Terror,http://store.steampowered.com/app/498450/A_Wee...,2016-08-05,"[Indie, Horror, Point & Click]",http://steamcommunity.com/app/498450/reviews/?...,[Single-player],4.99,0.0,498450.0,Richard Haraším
98584,None,"[Action, Casual, Indie, Simulation]",Solaroids - Soundtrack (OST),Solaroids - Soundtrack (OST),http://store.steampowered.com/app/652440/Solar...,2017-06-22,"[Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/652440/reviews/?...,"[Single-player, Multi-player, Local Multi-Play...",2.99,0.0,652440.0,DynF/X Digital
99161,Kitfox Games,"[Indie, Simulation]",The Shrouded Isle,The Shrouded Isle,http://store.steampowered.com/app/501320/The_S...,2017-08-04,"[Indie, Simulation, Management, Lovecraftian, ...",http://steamcommunity.com/app/501320/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",9.99,0.0,501320.0,"Jongwoo Kim,Erica Lahaie,FX Bilodeau,Tanya Short"
107144,Dovetail Games - Trains,[Simulation],Train Simulator: GWR Nunney Castle Steam Loco ...,Train Simulator: GWR Nunney Castle Steam Loco ...,http://store.steampowered.com/app/513110/Train...,2017-03-09,[Simulation],http://steamcommunity.com/app/513110/reviews/?...,"[Single-player, Downloadable Content, Steam Ac...",10.99,0.0,513110.0,Bossman Games
111065,OtakuMaker.com,"[Action, Indie]",1917 - The Alien Invasion DX,1917 - The Alien Invasion DX,http://store.steampowered.com/app/470060/1917_...,2016-06-10,"[Shoot 'Em Up, Action, Indie, Singleplayer, 2D...",http://steamcommunity.com/app/470060/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",4.99,0.0,470060.0,Andrade Games
103498,None,"[Action, Adventure, Indie, Racing, Simulation]",Road Redemption: Origins of Road Redemption Video,Road Redemption: Origins of Road Redemption Video,http://store.steampowered.com/app/727440/Road_...,2017-10-03,"[Action, Adventure, Indie, Simulation, Racing]",http://steamcommunity.com/app/727440/reviews/?...,"[Single-player, Multi-player, Local Multi-Play...",9.99,0.0,727440.0,"Pixel Dash Studios, EQ"
98429,CivilSavages,"[Action, Indie, Early Access]",Goodbye My King,Goodbye My King,http://store.steampowered.com/app/647410/Goodb...,2017-06-16,"[Early Access, Indie, Action, Horror]",http://steamcommunity.com/app/647410/reviews/?...,[Single-player],6.99,1.0,647410.0,CivilSavages
101542,None,None,Fireplace For Your Home: Holiday Music Edition,Fireplace For Your Home: Holiday Music Edition,http://store.steampowered.com/app/770190/Firep...,2017-12-15,[Movie],http://steamcommunity.com/app/770190/reviews/?...,[Captions available],3.99,0.0,770190.0,None


In [170]:

#ruta del archivo JSON comprimido
ruta_archivo = '.\\data\\original\\user_reviews.json.gz'
#inicializo una lista para ir guardando los diccionarios de cada lista
filas=[]

# con la funcion gzip abro el archivo y lo recorro con un for por todas sus lineas
with gzip.open(ruta_archivo, 'rt', encoding='latin-1') as file:
    for line in file.readlines():
        #con ast.literal_eval se confierte la linea en un diccionario
        diccionario = ast.literal_eval(line)
        filas.append(diccionario)

user_review = pd.DataFrame(filas)

In [171]:
user_review

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [172]:
user_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


In [173]:
user_review.columns

Index(['user_id', 'user_url', 'reviews'], dtype='object')

In [174]:
user_review.loc[0, 'reviews']

[{'funny': '',
  'posted': 'Posted November 5, 2011.',
  'last_edited': '',
  'item_id': '1250',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.'},
 {'funny': '',
  'posted': 'Posted July 15, 2011.',
  'last_edited': '',
  'item_id': '22200',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': "It's unique and worth a playthrough."},
 {'funny': '',
  'posted': 'Posted April 21, 2011.',
  'last_edited': '',
  'item_id': '43110',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!'}]

In [175]:
user_review.loc[10, 'reviews']

[{'funny': '',
  'posted': 'Posted June 11.',
  'last_edited': '',
  'item_id': '440',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'mt bom'},
 {'funny': '',
  'posted': 'Posted August 25, 2014.',
  'last_edited': 'Last edited July 30, 2015.',
  'item_id': '304930',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Ã\x89 muito bom'}]

In [176]:
type(user_review['reviews'])

pandas.core.series.Series

In [177]:
#El contenido de la columna reviews es una serie de pandas, con explode 
#aplano ese contenido y genero una nueva fila con repitiendo el contenido de las otras columnas
user_review=user_review.explode('reviews')

In [178]:
#ahora obtengo un nuevo DF de 59333 filas
user_review.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59333 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   59333 non-null  object
 1   user_url  59333 non-null  object
 2   reviews   59305 non-null  object
dtypes: object(3)
memory usage: 1.8+ MB


In [179]:
user_review

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20..."
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011...."
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011..."
1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014...."
1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2..."
...,...,...,...
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 10.', 'la..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 8.', 'las..."
25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '1 person found this review funny', ..."
25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '', 'posted': 'Posted July 20.', 'la..."


In [180]:
user_review['reviews'].head(1)

0    {'funny': '', 'posted': 'Posted November 5, 20...
Name: reviews, dtype: object

In [181]:
user_review=pd.concat([user_review.drop(['reviews'],axis=1),user_review['reviews'].apply(pd.Series)],axis=1)  

In [182]:
user_review.sample(5)

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review,0
1201,ysarge,http://steamcommunity.com/id/ysarge,,"Posted August 22, 2014.","Last edited November 24, 2014.",281370,No ratings yet,False,Played for 2 minutes and closed it.Steam still...,NaN
10162,TitlerBitch,http://steamcommunity.com/id/TitlerBitch,,"Posted January 3, 2014.",,12110,1 of 1 people (100%) found this review helpful,True,"Best game in the series, have spent many hours...",NaN
10426,76561198085327867,http://steamcommunity.com/profiles/76561198085...,,"Posted March 20, 2014.",,218620,1 of 2 people (50%) found this review helpful,True,"I strongly recommend this game, it's a brillia...",NaN
4679,76561198011735059,http://steamcommunity.com/profiles/76561198011...,,"Posted July 10, 2012.",,212180,0 of 1 people (0%) found this review helpful,True,meh,NaN
2149,Singularity_iOS,http://steamcommunity.com/id/Singularity_iOS,1 person found this review funny,"Posted October 5, 2015.",,17520,No ratings yet,True,Makes one of the best games of all time co-op....,NaN


In [183]:
user_review.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59333 entries, 0 to 25798
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   user_id      59333 non-null  object 
 1   user_url     59333 non-null  object 
 2   funny        59305 non-null  object 
 3   posted       59305 non-null  object 
 4   last_edited  59305 non-null  object 
 5   item_id      59305 non-null  object 
 6   helpful      59305 non-null  object 
 7   recommend    59305 non-null  object 
 8   review       59305 non-null  object 
 9   0            0 non-null      float64
dtypes: float64(1), object(9)
memory usage: 5.0+ MB


In [184]:
#Analisis de sentimientos usando la columna review

# primero se descarga un diccionario para realizar el analisis (vader_lexicon)

nltk.download('vader_lexicon')

# instancio SentimentIntensityAnalyzer
modelo = SentimentIntensityAnalyzer()

# defino la función para realizar el análisis
def analizador(texto):
    # Obtener el puntaje de sentimiento usando SentimentIntensityAnalyzer
    sentimiento_score = modelo.polarity_scores(texto)
    
    # reviso que el texto no este vacio
    if texto and not pd.isnull(texto):
        # el rango de score va de -1 a 1, por lo tanto decido dividir los 3 grupos en rangos
        # similares (0.66 c/u), de -1 a -0.33, de -0.33 a 0.33 y de 0.33 a 1
        if sentimiento_score['compound'] >= 0.33:
            return 2  # Sentimiento positivo
        elif sentimiento_score['compound'] >= -0.33:
            return 1  # Sentimiento neutral si estan entre -0.33 y 0.33
        else:
            return 0  # Sentimiento negativo
    else:
        return 1  # Sentimiento neutro si esta vacio el texto


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Casa\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [185]:
user_review.isna().sum()

user_id            0
user_url           0
funny             28
posted            28
last_edited       28
item_id           28
helpful           28
recommend         28
review            28
0              59333
dtype: int64

In [186]:
user_review.dropna(subset=['review'], inplace=True)

In [187]:
user_review.isna().sum()

user_id            0
user_url           0
funny              0
posted             0
last_edited        0
item_id            0
helpful            0
recommend          0
review             0
0              59305
dtype: int64

In [188]:
user_review.drop(columns=['user_url','funny','helpful','last_edited',0],inplace=True)

In [189]:
user_review['sentiment_analysis']  = user_review['review'].apply(analizador)
user_review.drop(columns='review',inplace=True) # Aplica la funcion que determina el sentimiento

In [190]:
user_review

,user_id,posted,item_id,recommend,sentiment_analysis
0,76561197970982479,"Posted November 5, 2011.",1250,True,2
0,76561197970982479,"Posted July 15, 2011.",22200,True,1
0,76561197970982479,"Posted April 21, 2011.",43110,True,2
1,js41637,"Posted June 24, 2014.",251610,True,2
1,js41637,"Posted September 8, 2013.",227300,True,2
...,...,...,...,...,...
25797,76561198312638244,Posted July 10.,70,True,2
25797,76561198312638244,Posted July 8.,362890,True,2
25798,LydiaMorley,Posted July 3.,273110,True,2
25798,LydiaMorley,Posted July 20.,730,True,2


In [191]:
user_review['sentiment_analysis'].value_counts()

sentiment_analysis
2    33909
1    18354
0     7042
Name: count, dtype: int64

In [192]:
user_review.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59305 entries, 0 to 25798
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             59305 non-null  object
 1   posted              59305 non-null  object
 2   item_id             59305 non-null  object
 3   recommend           59305 non-null  object
 4   sentiment_analysis  59305 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 2.7+ MB
